In [1]:
import CMUTweetTagger

In [2]:
import pickle

In [3]:
from nltk.tokenize import TweetTokenizer

In [4]:
import nltk
nltk.__version__

'3.2.1'

In [5]:
from gensim import corpora, models, similarities

from sklearn import metrics
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import adjusted_mutual_info_score

from nltk import Text
from nltk.tokenize import line_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.probability import *
from nltk import bigrams, trigrams, ngrams

from collections import Counter
from operator import itemgetter
from numpy import array
import numpy as np
import time
import re
import string
import json
import sys

/Users/ynh3/clustering/algo_test/lib/python2.7/site-packages/gensim-0.10.2-py2.7-macosx-10.9-intel.egg/gensim/__init__.py:10: UserWarning: Module gensim was already imported from /Users/ynh3/clustering/algo_test/lib/python2.7/site-packages/gensim-0.10.2-py2.7-macosx-10.9-intel.egg/gensim/__init__.pyc, but /Users/ynh3/clustering/algo_test/bin is being added to sys.path
  __version__ = __import__('pkg_resources').get_distribution('gensim').version


In [151]:
f1 = 'statuses.log.2015-04-08-00'
fpath = '/Users/ynh3/Data/Tweets_2015/'
#fpath = '/Users/ynh3/Data/Tweets_2015_rt/'

###This reads values from the asci file and saves the actual tweets in tweets_table
file = open(fpath + f1)
i = 0
tweets_table={}
for row in file:
    tweet = row.strip()
    tweet_text = tweet.split(' ',0)[0]

    list_tweet=[]
    list_tweet.append(tweet_text)
    i=i+1
    tweets_table[i]=list_tweet

values = tweets_table.viewvalues()
file.close()
temp = list(values)
tweets = [x[0] for x in temp]
print "Done with generating tweets_table"

Done with generating tweets_table


In [152]:
len(tweets)

36419

In [153]:
tweets_url= [(i,t) for i,t in enumerate(tweets) if "http://" in t or "https://" in t]
len(tweets_url)

10803

In [154]:
tweets_url[0]

(3, 'Pray for us sinners #mother https://t.co/iKfL8GPvQ5')

In [61]:
tweets[3]

'RT @16KVLO: My new sounds: Soul Musiq(Prod. Yung Prodigy &amp; Bree Honcho) https://t.co/w4ueXKkL7H on #SoundCloud'

In [62]:
#percentage
float(len(tweets_url))/float(len(tweets))

0.46738391393032835

In [155]:
tweets_url_value = [tw[1].replace('\r', ' ') for tw in tweets_url]
#tweets_cleaned = [tw.replace('\r', ' ') for tw in s]
len(tweets_url_value)

10803

In [64]:
tweets_url_value[0]

'RT @AkanshaGautam_: Little lady by Camelia Mot @SusanGilbert @Cynthiapoet @drkent @_Akanshagautam http://t.co/8Cm4xMaPKd'

In [156]:
tagged_list = CMUTweetTagger.runtagger_parse(tweets_url_value)
#tweets_entity = [y for y in tagged_list for e in y if e[1]=='N' or e[1]=='^']
len(tagged_list)

10803

In [157]:
tweets_entity=[]
#keep a list of actual tweets
tweets_list=[]
for y in tagged_list:
    for e in y:
        if e[1]=='^':
            tweets_entity.append(e[0])
            i = tagged_list.index(y)
            tweets_list.append(tweets_url[i][1])
            break

In [158]:
len(tweets_entity)

6026

In [159]:
tweets_entity[:1]

['#badoo']

In [160]:
tweets_list[:5]

['As one of the hottest people on #badoo, I can show off about it all week on http://t.co/7GWsLXJZIF http://t.co/SZ0UEgiO2w',
 'Nexxen NE8801M 2T/SIL/BLK http://t.co/lDHzuxzgoB #Nexxen',
 'Wcw goes out to my fire baby http://t.co/BmFeek6AeD',
 'I posted a new photo to Facebook http://t.co/mqZnUx0Sj4',
 'A decent afternoon in Naches for a ZHS track meet... http://t.co/digwlNEw1Q']

In [25]:
len(tweets_list)

6027

In [26]:
len(tweets_entity)

6027

In [27]:
len(tagged_list)

10805

In [28]:
#percentage
float(len(tweets_list))/float(len(tweets))

0.16549054065185756

In [29]:
#percentage
float(len(tweets_list))/float(len(tweets_url))

0.5577973160573808

In [78]:
tweets_list[:10]

['RT @AkanshaGautam_: Little lady by Camelia Mot @SusanGilbert @Cynthiapoet @drkent @_Akanshagautam http://t.co/8Cm4xMaPKd', 'RT @ThePlayersWorld: Cinque terre, Italy villas and scenery http://t.co/CmM419fDuU', 'RT @16KVLO: My new sounds: Soul Musiq(Prod. Yung Prodigy &amp; Bree Honcho) https://t.co/w4ueXKkL7H on #SoundCloud', 'RT @cuckoldforcedbi: http://t.co/wMCFtFoxPB Dana Vespoli Gets Naughty! http://t.co/bPpzUbObty', "RT @Mandisa09150378: I hate you Perrie you made my life a mess because of you zayn can't be in 1D I'm going to unfollow you http://t.co/ORK", 'RT @Ashton5SOS: Played Cowboys for a day out in the desert  http://t.co/FaRGd2fxt3', 'RT @75blinkhes: NIALL AND SPORTS IS SO CUTE EVEN THOUGH HIS UPDATES BORE US IT MAKES HIM HAPPY AND I LOVE HIM http://t.co/1ym6pq3WXg', 'RT @AmaZeGoD: Me and my duo @CRUClFIED puttin im that work tonight. Even though we lost it was a really GG. @MLGTechh @RiftLegacy http://t.', 'RT @GovChristie: What does a Girl Scout, the  NYTimes and the cur

In [75]:
%pprint

Pretty printing has been turned OFF


In [161]:
entity_enumerate = [e for e in set(tweets_entity)]
len(entity_enumerate)

3690

In [162]:
entity_enumerate[:10]

['SPx',
 'Savage',
 'En',
 '#3D',
 'Kensington',
 'EliteField',
 'Thalassaemia',
 '#EURUSD',
 'Olympics',
 '#supernatural']

In [163]:
stoplist = set([line.replace("\n","") for line in open('/Users/ynh3/Data/stopword-list.txt')])
pun = [',', '-','.', ':', '(', ')', '--', ';', '...', 'may', 'must', 'us', 'via','a','the', 'rt', 'gg', 'gt', 'lt', 'la', 'de', 'te', 'lol', 'follow', 'followers', 'unfollow', 'unfollowers', 'unfollower', 'follower']

In [165]:
    ###TOKENIZING
    tknzr = TweetTokenizer()   
    tweet_count = 0
    tweet_tokens=[]
    new_tweet_list=[]
    #regex and then line_tokenize, so each line is one tweet (one problem: two line tweets will become two tweets.)
    i = 0
    
    for line in tweets_list:
  
        new_l = tknzr.tokenize(line)
        lower_l = [e.lower() for e in new_l]
        new_ll = [e for e in lower_l if e not in stoplist and e not in pun and e.isalpha() and len(e) > 2]

        if len(new_ll) > 0:
            tweet_tokens.append(new_ll)
            tweet_count += 1
            new_tweet_list.append(line)
            #new_list=[]
            #new_list.append(new_ll)
            #tweets_list.append(new_list)'''
        i += 1
    print "Done with tokenizations. tweets count: " , tweet_count

Done with tokenizations. tweets count:  6010


In [166]:
###GENSIM MODELING AND MAKING DICTIONARY
dictionary = corpora.Dictionary(line.lower().split() for line in new_tweet_list)
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
#once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.iteritems() if docfreq < 5]
dictionary.filter_tokens(stop_ids) 
dictionary.compactify() # remove gaps in id sequence after words that were removed

In [167]:
   ###LSI TOPIC MODELING BASED ON OPT
class MyCorpus(object):
    def __iter__(self):
        for line in tweet_tokens:
            # assume there's one document per line
            yield dictionary.doc2bow(line)

gensim_corpus = MyCorpus()

tfidf = models.TfidfModel(gensim_corpus)
corpus_tfidf = tfidf[gensim_corpus]


In [168]:
    n_cluster = 50
    # LSI computing
    t1=time.time()
    lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=n_cluster) # initialize an LSI transformation
    corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
    t2 = time.time()
    print 'LSI model takes {} seconds'.format(t2 - t1)

    # Creating LSI Label for each document
    lsi_topic_labels=[]
    i = 0
    j = 0

    t1=time.time()
    #topicmax is numpy.ndarray
    for i,doc in enumerate(lsi[corpus_tfidf]):
        a = np.array(doc)
        absA= abs(a)
        if (len(a)==0):
            cluster_id=0
            prob_value=0.0
            j=j+1
        else:
            try:
                topicmax = absA.argmax(axis=0)
            except ValueError:
                print "ValueError at"
                print i
                #print a
            cluster_id = a[topicmax.item(1)][0]
            prob_value = abs(a[topicmax.item(1)][1])

        triple = i, cluster_id, prob_value
        lsi_topic_labels.append(triple)
        i=i+1
    t2=time.time()
    print 'lsi is taking {} seconds'.format(t2-t1)

LSI model takes 2.751060009 seconds
lsi is taking 0.60023021698 seconds


In [169]:
    topics = []
    for i in range(0, n_cluster):
        l = lsi.print_topic(i,topn=5)
        prob_list = [(abs(float(a[:5])),a[6:])  for a in (l.split('+ ')) if abs(float(a[:5])) > 0.3]
        sorted(prob_list)
        topics.append(prob_list)


    ###MAKING READY FOR
    cluster_topic_list = sorted(lsi_topic_labels,key=itemgetter(1,2), reverse=True)

    cluster_list = sorted(lsi_topic_labels,key=lambda x: x[1])
    counter = Counter(b for a,b,c in cluster_list)
    counter_sort = counter.most_common(n_cluster)
    cid_sort = [e[0] for e in counter_sort]

    print 'length of lsi_topic_labels is {}'.format(len(lsi_topic_labels))
    print 'length of cluster_list is {}'.format(len(cluster_list))
    print 'length of counter_sort is {}'.format(len(counter_sort))
    print 'n_cluster is {}'.format(n_cluster)
    print 'cid_sort is {}'.format(len(cid_sort))


length of lsi_topic_labels is 6010
length of cluster_list is 6010
length of counter_sort is 50
n_cluster is 50
cid_sort is 50


In [102]:
%pprint

Pretty printing has been turned ON


In [103]:
topics[:10]

[[(0.41, u'*"twitter" '),
  (0.37, u'*"click" '),
  (0.37, u'*"want" '),
  (0.36, u'*"meet" '),
  (0.36, u'*"girls"')],
 [(0.59, u'*"paul" '), (0.58, u'*"walker" '), (0.53, u'*"rip" ')],
 [(0.45, u'*"spots" '),
  (0.45, u'*"surf" '),
  (0.45, u'*"bali" '),
  (0.45, u'*"beginners" '),
  (0.42, u'*"best"')],
 [(0.71, u'*"outfit" '), (0.69, u'*"point" ')],
 [(0.449, u'"steroid" '),
  (0.449, u'"gaining" '),
  (0.449, u'"edge" '),
  (0.446, u'"athletes" '),
  (0.441, u'"legal"')],
 [(0.78, u'*"like" '), (0.33, u'*"fav" ')],
 [(0.498, u'"hair" ')],
 [(0.53, u'*"instagram" '), (0.34, u'*"likes" ')],
 [(0.576, u'"mom" '), (0.576, u'"bitmoji" '), (0.576, u'"discovers" ')],
 [(0.41, u'*"cutie" '),
  (0.41, u'*"ceramic" '),
  (0.41, u'*"africanurbanart" '),
  (0.41, u'*"giraffe" '),
  (0.4, u'*"african"')]]

In [104]:
lsi_topic_labels[0]

(0, 49.0, 0.0154188733599947)

In [107]:
cluster_list[1]

(38, 0.0, 0.063730617292470565)

In [119]:
###WRITING JSON FOR HASH
def jdefault(o):
    return o.__dict__

output_file = open(fpath + 'tweet-topic.txt', 'w')
#json_file = open(jpath + f1 + '-'+ opt + 'wo-cw-ch.json', 'w')
#topic_lsi_file = open(cpath + f1 + opt + '.cluster', 'w')
data_list=[]
data = {}
count=0
i=0

for i in range(0, len(cid_sort)):
    sublist = [e for e in cluster_topic_list if e[1] == cid_sort[i]]
    
    t_index = int(cid_sort[i])
    topic = topics[t_index]
    string = [x[1] for x in topic]
    string = ' '.join(string)
    topic_str = string.replace("\"","")
    output_file.write(topic_str)
    output_file.write('-----------------------\n')
    
    for j in range(0,len(sublist)):
        data = {}
        index = sublist[j][0]
        if (index >= len(new_tweet_list)):
            print index, i, j
            break
        tweet_str = new_tweet_list[index]
        
        data['id']=int(count+1)
        data['topic']=topic_str
        data['tweet']=tweet_str
        data_list.append(data)
        #k = k+1
        count=count+1
        output_file.write(tweet_str+'\n')

        count_str = str(count)

#json_file.write('{"total":')
#json_file.write(count_str)
#json_file.write(',')
#json_file.write('"tweets": [')

'''for p in range(0,len(data_list)):
    record = data_list[p]
    json_data = json.dumps(record, default=jdefault)
    json_file.writelines(json_data)
    if (p < len(data_list) - 1 ):
        json_file.write(',')

json_file.write(']}')
json_file.close()'''
print("Total number of tweets is {}".format(count))
output_file.close()

   

Total number of tweets is 4561


In [114]:
data_list[:10]

[{'id': 1,
  'topic': u'shri ',
  'tweet': 'RT @MIB_India: PM Shri @narendramodi launches the logo of #MUDRA http://t.co/X9QVLHr5iQ'},
 {'id': 2,
  'topic': u'shri ',
  'tweet': 'RT @MIB_India: PM Shri @narendramodi launches the logo of #MUDRA http://t.co/X9QVLHr5iQ'},
 {'id': 3,
  'topic': u'shri ',
  'tweet': 'RT @MIB_India: PM Shri @narendramodi launches the logo of #MUDRA http://t.co/X9QVLHr5iQ'},
 {'id': 4,
  'topic': u'shri ',
  'tweet': 'RT @MIB_India: PM Shri @narendramodi launches the logo of #MUDRA http://t.co/X9QVLHr5iQ'},
 {'id': 5,
  'topic': u'shri ',
  'tweet': 'RT @MIB_India: PM Shri @narendramodi launches the logo of #MUDRA http://t.co/X9QVLHr5iQ'},
 {'id': 6,
  'topic': u'shri ',
  'tweet': 'RT @MIB_India: PM Shri @narendramodi launches the logo of #MUDRA http://t.co/X9QVLHr5iQ'},
 {'id': 7,
  'topic': u'shri ',
  'tweet': 'RT @MIB_India: PM Shri @narendramodi launches the logo of #MUDRA http://t.co/X9QVLHr5iQ'},
 {'id': 8,
  'topic': u'shri ',
  'tweet': 'RT @MIB_Ind

In [131]:
topic_list = [x[1] for x in y for y in topics]
topic_set = set(topic_list)
topic_set[:10]

TypeError: 'set' object has no attribute '__getitem__'

In [132]:
topic_list = [x[i][1] for x in topics for i in range(0,len(x))]

In [134]:
topic_list[:10]

[u'*"twitter" ',
 u'*"click" ',
 u'*"want" ',
 u'*"meet" ',
 u'*"girls"',
 u'*"paul" ',
 u'*"walker" ',
 u'*"rip" ',
 u'*"spots" ',
 u'*"surf" ']

In [170]:
l = []
t=[]
for x in topics:
    m = [x[i][1] for i in range(0,len(x))]
    if (len(m)>0):
        t.append(m)
        
            

In [171]:
t[:50]

[[u'*"weather" ', u'*"update" '],
 [u'*"new" ', u'*"week" ', u'*"twitter" ', u'*"posted" ', u'*"facebook"'],
 [u'"posted" ', u'"facebook" ', u'"photo" ', u'*"twitter" ', u'*"week"'],
 [u'"apple" ', u'"watch" '],
 [u'"vote" ', u'"trevor" '],
 [u'*"april" '],
 [u'"collected" ', u'"coins" ', u'"gold" '],
 [u'"read" ', u'"full" ', u'"ebay" ', u'*"apple" '],
 [u'*"here" ', u'*"video" '],
 [u'*"harvested" ', u'*"food" '],
 [u'*"see" ', u'*"who" '],
 [u'"video" ', u'*"here" ', u'"liked" '],
 [u'"now" ', u'"online" '],
 [u'*"love" ', u'*"now" '],
 [u'"love" ', u'*"black" '],
 [u'*"love" ', u'"now" ', u'*"black" '],
 [u'"earning" '],
 [u'*"live" ', u'*"like" '],
 [u'"free" ', u'*"live" '],
 [u'*"live" ', u'"like" '],
 [u'*"just" ', u'"free" '],
 [u'"camgirl" ', u'"twitter" '],
 [u'*"people" ', u'*"followed" ', u'"twitter" '],
 [u'*"people" '],
 [u'"camgirl" '],
 [u'"click" '],
 [u'*"set" '],
 [u'"will" '],
 [u'"can" ', u'"check" '],
 [u'"officer" '],
 [u'"apr" ', u'"leader" ', u'"wed" ', u'"bur

In [143]:
len(topics)

50